In [1]:
import numpy as np
import pandas as pd
from equiflow import EquiFlow

In [2]:
np.random.seed(42)
n = 100000
data = {
    'age': np.random.randint(18, 80, size=n),
    'sofa': np.random.choice([0,1,2,3,4,5,6,7,8,9,10,15, np.nan], size=n),
    'race': np.random.choice(['White', 'Black', 'Asian', 'Hispanic', None],
                             size=n),
    'sex': np.random.choice(['Male', 'Female'], size=n),
    'english': np.random.choice(['Fluent', 'Limited', np.nan, None], size=n),
}

for i in range(1, 11):
    data[f'var{i}'] = np.random.randn(n)

df = pd.DataFrame(data)

In [3]:
data_0 = df.copy()
data_1 = data_0.loc[data_0.english.notnull()]
data_2 = data_1.loc[data_1.sofa.notnull()]

ef = EquiFlow(dfs = [data_0, data_1, data_2])



In [4]:
ef.table_flows(
    label_suffix=True,
    thousands_sep=True,
)

Cohort Flow,0 to 1,1 to 2
,,
"Inital, n","100,000","50,022"
"Removed, n","49,978","3,874"
"Result, n","50,022","46,148"


In [10]:
t = ef.table_characteristics(
    categorical = ['race','sex', 'english'],
    nonnormal = ['sofa'],
    normal = ['age'],
    format_cat = 'N (%)',
    format_cont = 'Mean ± SD',
    missingness = True,
    decimals = 1,
    label_suffix = True,
    thousands_sep = False,
    rename={'race': 'Race and Ethnicity',
            'english': 'English Proficiency',
            'sex':'Sex',
            'sofa': 'SOFA',
            'age': 'Age',  
            }
)
display(t)

Cohort                  \
                                                  0               1   
Variable                   Value                                      
Overall                                      100000           50022   
Race and Ethnicity, N (%)  Hispanic    20024 (20.0)    10011 (20.0)   
                           Asian       19952 (20.0)     9918 (19.8)   
                           Black       20066 (20.1)    10088 (20.2)   
                           White       19931 (19.9)     9978 (19.9)   
                           Missing     20027 (20.0)    10027 (20.0)   
Sex, N (%)                 Male        50052 (50.1)    25049 (50.1)   
                           Female      49948 (49.9)    24973 (49.9)   
                           Missing          0 (0.0)         0 (0.0)   
English Proficiency, N (%) Fluent      25134 (25.1)    25134 (50.2)   
                           Limited     24888 (24.9)    24888 (49.8)   
                           Missing     49978 (50.0)         0 (0.0)   
Age, Mean ± SD                          48.5 ± 17.9     48.5 ± 17.9   
                           Missing          0 (0.0)         0 (0.0)   
SOFA, Median [IQR]                   6.0 [3.0, 9.0]  6.0 [3.0, 9.0]   
                           Missing       7812 (7.8)      3874 (7.7)   

                                                     
                                                  2  
Variable                   Value                     
Overall                                       46148  
Race and Ethnicity, N (%)  Hispanic     9197 (19.9)  
                           Asian        9145 (19.8)  
                           Black        9323 (20.2)  
                           White        9218 (20.0)  
                           Missing      9265 (20.1)  
Sex, N (%)                 Male        23089 (50.0)  
                           Female      23059 (50.0)  
                           Missing          0 (0.0)  
English Proficiency, N (%) Fluent      23223 (50.3)  
                           Limited     22925 (49.7)  
                           Missing          0 (0.0)  
Age, Mean ± SD                          48.5 ± 17.9  
                           Missing          0 (0.0)  
SOFA, Median [IQR]                   6.0 [3.0, 9.0]  
                           Missing          0 (0.0)

In [6]:
t.loc[('Sex, N (%)', 'Male'), ('Cohort',0)] 

'50052 (50.1)'

In [7]:
t.loc[('Sex, N (%)', 'Male'), ('Cohort',1)] 


'25049 (50.1)'

In [8]:
t = ef.table_characteristics(
    categorical = ['race','sex', 'english'],
    nonnormal = ['sofa'],
    normal = ['age'],
    format = 'N',
    missingness = True,
    decimals = 1,
    label_suffix = True,
    rename={'race': 'Race and Ethnicity',
            'english': 'English Proficiency',
            'sex':'Sex',
            'sofa': 'SOFA',
            'age': 'Age',  
            }
)
display(t)

TypeError: BaseTable.__init__() got an unexpected keyword argument 'format'

In [ ]:
t.loc[('Sex, N', 'Male'), ('Cohort',0)]

'50,052'

In [ ]:
prop1 = [0.501]
prop1[1:]

[]